# Try to train on PNC age data and predict fibromyalgia data age
Will LatSim learn any relevant features?

In [38]:
# Get the fibromyalgia fc data

import pickle

fibrotsname = '/home/anton/Documents/Tulane/Research/ImageNomeR/data/fmri-FC-slim.pkl'

with open(fibrotsname, 'rb') as f:
    fibrodict = pickle.load(f)
    
print(list(fibrodict.keys()))

['FC-slim', 'subjNum2IdxMap', 'subjIdx2NumMap', 'groupsNormalDiagMap']


In [50]:
import pandas
import numpy as np

csvname = '/home/anton/Documents/Tulane/Hackathon/Clinical_fm_66_.csv'

csv = pandas.read_csv(csvname)
fibroage = []
fibrofc = []

for i in fibrodict['subjIdx2NumMap'].keys():
    rid = int(fibrodict['subjIdx2NumMap'][i])
    age = csv.loc[csv['rid'] == rid]['1_age']
    if len(age) > 0:
        fibroage.append(int(age))
        fibrofc.append(fibrodict['FC-slim'][i])
        
fibroage = np.stack(fibroage)
fibrofc = np.stack(fibrofc)

print(fibroage.shape)
print(fibrofc.shape)

(66,)
(66, 34716)


In [129]:
import torch
from sklearn.decomposition import PCA

ntrain = 60

pca = PCA(n_components=10).fit(fibrofc)
fb = pca.transform(fibrofc)

x = torch.from_numpy(np.concatenate([fb, np.ones((fb.shape[0], 1))], axis=1)).float().cuda()
xtr = x[:ntrain]
xt = x[ntrain:]

y = torch.from_numpy(fibroage).float().cuda()
ytr = y[:ntrain]
yt = y[ntrain:]

w, _, _, _ = torch.linalg.lstsq(xtr.T@xtr+0.0001*torch.eye(11).float().cuda(), xtr.T@ytr)

print(torch.mean((yt-torch.mean(ytr))**2)**0.5)
print(torch.mean((yt-xt@w)**2)**0.5)

tensor(2.5609, device='cuda:0')
tensor(3.6682, device='cuda:0')


In [69]:
import sys

sys.path.append('../../LatentSimilarity')

from latsim import LatSim

print('Complete')

Complete


In [156]:
import torch.nn as nn

ntrain = 20

x = torch.from_numpy(fibrofc).unsqueeze(1).float().cuda()
xtr = x[:ntrain]
xt = x[ntrain:]

y = torch.from_numpy(fibroage).float().cuda()
ytr = y[:ntrain]
yt = y[ntrain:]

# mseLoss = nn.MSELoss()

# nEpochs = 1000
# pPeriod = 100

# sim = LatSim(1, x, dp=0.5, edp=0.1, wInit=1e-4, dim=10, temp=1)

# optim = torch.optim.Adam(sim.parameters(), lr=1e-4, weight_decay=1e-3)
# sched = torch.optim.lr_scheduler.ReduceLROnPlateau(optim, patience=10, factor=0.9, eps=1e-7)

# for epoch in range(nEpochs):
#     optim.zero_grad()
#     yhat = sim(xtr, [ytr])[0][0]
#     loss = mseLoss(yhat, ytr)
#     loss.backward()
#     optim.step()
#     sched.step(loss)
#     if epoch % pPeriod == 0 or epoch == nEpochs-1:
#         print(f'{epoch} recon: {loss**0.5} {sched._last_lr}')

# print('Complete')

sim.eval()
yhat = sim(x, [y])[0][0][ntrain:]
loss = mseLoss(yhat, yt)**0.5
print(float(loss))

tensor(5.8076, device='cuda:0')
5.7591047286987305


In [134]:
# Using newly preprocessed subjects

import pickle

metadictname = '/home/anton/Documents/Tulane/Research/PNC_Good/PNC_agesexwrat.pkl'
alltsname = '/home/anton/Documents/Tulane/Research/PNC_Good/PNC_PowerTS_float2.pkl'

with open(metadictname, 'rb') as f:
    metadict = pickle.load(f)

with open(alltsname, 'rb') as f:
    allts = pickle.load(f)
    
print(list(metadict.keys()))
print(list(allts.keys()))
print('Complete')

['age', 'sex', 'wrat', 'missingage', 'missingsex', 'missingwrat', 'failedqc']
['emoid', 'nback', 'rest']
Complete


In [135]:
'''
Get subjects that have all tasks and paras specified
Functions for creating independent and response variables
'''

import numpy as np

def get_subs(allts, metadict, tasks, paras):
    # Get subs for all paras
    for i,para in enumerate(paras):
        tmpset = set([int(sub[4:]) for sub in allts[para].keys()])
        if i == 0:
            paraset = tmpset
        else:
            paraset = paraset.intersection(tmpset)
    # Get subs for all tasks
    for i,task in enumerate(tasks):
        tmpset = set([sub for sub in metadict[task].keys()])
        if i == 0:
            taskset = tmpset
        else:
            taskset = paraset.intersection(tmpset)
    # Remove QC failures
    allsubs = taskset.intersection(paraset)
    for badsub in metadict['failedqc']:
        try:
            allsubs.remove(int(badsub[4:]))
        except:
            pass
    return allsubs

def get_X(allts, paras, subs):
    X = []
    for para in paras:
        pX = [allts[para][f'sub-{sub}'] for sub in subs]
        pX = np.stack(pX)
        X.append(pX)
    return X

def get_y(metadict, tasks, subs):
    y = []
    for task in tasks:
        if task == 'age' or task == 'wrat':
            var = [metadict[task][sub] for sub in subs]
            var = np.array(var)
            y.append(var)
        if task == 'sex':
            maleness = [metadict[task][sub] == 'M' for sub in subs]
            maleness = np.array(maleness)
            sex = np.stack([maleness, 1-maleness], axis=1)
            y.append(sex)
    return y

subs = get_subs(allts, metadict, ['age'], ['rest', 'nback', 'emoid'])
print(len(subs))

X = get_X(allts, ['rest', 'nback', 'emoid'], subs)
print(X[0].shape)

847
(847, 264, 124)


In [137]:
# TS to condensed FC

from scipy import signal

def butter_bandpass(cutoff, fs, order=5):
    nyq = 0.5 * fs
    normal_cutoff = [cutoff[0] / nyq, cutoff[1] / nyq]
    b, a = signal.butter(order, normal_cutoff, btype='band', analog=False)
    return b, a

def butter_bandpass_filter(data, cutoff, fs, order=5):
    b, a = butter_bandpass(cutoff, fs, order=order)
    y = signal.filtfilt(b, a, data)
    return y

tr = 1.83

def filter_design_ts(X):
    Xs = []
    for i in range(X.shape[0]):
        nX = butter_bandpass_filter(X[i], [0.01, 0.2], 1/tr)
        Xs.append(nX)
    return np.stack(Xs)

def ts_to_flat_fc(X):
    p = np.corrcoef(X)
    a,b = np.triu_indices(p[0].shape[0], 1)
    p = p[a,b]
    return p

p = [np.stack([ts_to_flat_fc(ts) for ts in filter_design_ts(Xp)]) for Xp in X]
print(p[0].shape)

(847, 34716)


In [160]:
ntrain = 50

x = torch.from_numpy(p[0]).unsqueeze(1).float().cuda()
xtr = x[:ntrain]
xt = x[ntrain:]

y = get_y(metadict, ['age'], subs)[0]
y_t = torch.from_numpy(y).float().cuda()
ytr = y_t[:ntrain]
yt = y_t[ntrain:]

mseLoss = nn.MSELoss()

nEpochs = 1000
pPeriod = 100

sim = LatSim(1, x, dp=0.3, edp=0.1, wInit=1e-4, dim=1, temp=1)

optim = torch.optim.Adam(sim.parameters(), lr=1e-4, weight_decay=1e-4)
sched = torch.optim.lr_scheduler.ReduceLROnPlateau(optim, patience=10, factor=0.9, eps=1e-7)

for epoch in range(nEpochs):
    optim.zero_grad()
    yhat = sim(xtr, [ytr])[0][0]
    loss = mseLoss(yhat, ytr)
    loss.backward()
    optim.step()
    sched.step(loss)
    if epoch % pPeriod == 0 or epoch == nEpochs-1:
        print(f'{epoch} recon: {loss**0.5} {sched._last_lr}')

print('Complete')

sim.eval()
yhat = sim(x, [y_t])[0][0][ntrain:]
loss = mseLoss(yhat, yt)**0.5
print(float(loss))

0 recon: 35.21892166137695 [0.0001]
100 recon: 6.035109996795654 [8.1e-05]
200 recon: 4.178535461425781 [3.874204890000002e-05]
300 recon: 4.6053924560546875 [1.5009463529699922e-05]
400 recon: 3.9806344509124756 [7.1789798769185315e-06]
500 recon: 5.758667945861816 [2.7812838944369375e-06]
600 recon: 6.326016902923584 [1.0775263664305828e-06]
700 recon: 4.762712001800537 [9.697737297875246e-07]
800 recon: 3.31640887260437 [9.697737297875246e-07]
900 recon: 4.184307098388672 [9.697737297875246e-07]
999 recon: 4.126438617706299 [9.697737297875246e-07]
Complete
38.15674591064453


In [247]:
import itertools

import torch
import torch.nn as nn
import torch.nn.functional as F

class LowRankCodes(nn.Module):
    '''
    ranks: array of rank for each codebook matrix
    '''
    def __init__(self, ranks):
        super(LowRankCodes, self).__init__()
        self.As = []
        for rank in ranks:
            A = nn.Parameter(1e-2*torch.randn(rank,264).float().cuda())
            self.As.append(A)
        self.As = nn.ParameterList(self.As)

    def forward(self):
        book = []
        for A in self.As:
            AA = A.T@A
            book.append(AA)
        return torch.stack(book)
    
class LowRankWeights(nn.Module):
    '''
    For a single modality!
    
    nsubs: number of subjects
    ncodes: number of pages in the codebook
    nt: number of timepoints
    '''
    def __init__(self, nsubs, nmods, ncodes, nt):
        super(LowRankWeights, self).__init__()
        self.w = nn.Parameter(1e-2*torch.rand(nsubs, ncodes, nt).float().cuda())

    def forward(self, sub, book):
        w = self.w[sub]
        return torch.einsum('pt,pab->abt', w, book)
    
def get_recon_loss(x, xhat):
    return mseLoss(xhat, x)

def get_smooth_loss_fc(xhat):
    before = xhat[:,:,:-1]
    after = xhat[:,:,1:]
    return torch.mean((before-after)**2)

def get_sub_fc(subts):
    return torch.einsum('at,bt->abt',subts,subts)
    
# Timeseries
x = torch.from_numpy(ts[1]).float().cuda()
    
# Parameters
ntrain = 400
nbatch = 30
smooth_mult = 0.5
nEpochs = 50
pPeriod = 40

mseLoss = nn.MSELoss()
    
# Codebook and weights
lrc = LowRankCodes(300*[1])
ncodes = len(lrc.As)

lrw = LowRankWeights(ntrain, 1, ncodes, x.shape[-1])

# Optimizers
optim = torch.optim.Adam(itertools.chain(lrc.parameters(), lrw.parameters()), lr=1e-2, weight_decay=0)
sched = torch.optim.lr_scheduler.ReduceLROnPlateau(optim, patience=int(ntrain/nbatch)+5, factor=0.75, eps=1e-7)
    
for epoch in range(nEpochs):
    suborder = np.arange(ntrain)
#     np.random.shuffle(suborder)
    for bstart in range(0,ntrain,nbatch):
        bend = bstart+nbatch
        if bend > ntrain:
            bend = ntrain
        optim.zero_grad()
        book = lrc()
        recon_loss = 0
        smooth_loss_fc = 0
        for subidx in range(bstart, bend):
            sub = suborder[subidx]
            xsub = get_sub_fc(x[sub])
            xhat = lrw(sub, book)
            recon_loss += get_recon_loss(xsub, xhat)
            smooth_loss_fc += smooth_mult*get_smooth_loss_fc(xhat)
        recon_loss /= (bend-bstart)
        smooth_loss_fc /= (bend-bstart)
        totloss = recon_loss+smooth_loss_fc
        totloss.backward()
        optim.step()
        sched.step(totloss)
        if bstart % nbatch == 0:
            print(f'{epoch} {bstart} recon: {[float(ls)**0.5 for ls in [recon_loss, smooth_loss_fc]]} '
                  f'lr: {sched._last_lr}')

print('Complete')

0 0 recon: [0.006931878057287242, 5.054098581481911e-06] lr: [0.01]
0 30 recon: [0.007252615866164383, 5.069046498124369e-06] lr: [0.01]
0 60 recon: [0.007207318799926511, 5.0734193671780404e-06] lr: [0.01]
0 90 recon: [0.007554061423944718, 5.0869478431888015e-06] lr: [0.01]
0 120 recon: [0.008664171841262957, 5.099513035209801e-06] lr: [0.01]
0 150 recon: [0.007751345164208635, 5.11643584323018e-06] lr: [0.01]
0 180 recon: [0.00735320598659946, 5.132490740223583e-06] lr: [0.01]
0 210 recon: [0.007052520547498541, 5.151676294822728e-06] lr: [0.01]
0 240 recon: [0.008216108307432206, 5.1674755116092385e-06] lr: [0.01]
0 270 recon: [0.007459870688218595, 5.1904628279662914e-06] lr: [0.01]
0 300 recon: [0.006873753959932207, 5.213338638126864e-06] lr: [0.01]
0 330 recon: [0.007477348171786484, 5.234373735559109e-06] lr: [0.01]
0 360 recon: [0.007769009351238047, 5.2642294895775884e-06] lr: [0.01]
0 390 recon: [0.008794059177484395, 5.292160037638664e-06] lr: [0.01]
1 0 recon: [0.00693095

8 180 recon: [0.0063175406305075186, 0.001073335870150546] lr: [0.0075]
8 210 recon: [0.006104630428071503, 0.0010289049841461437] lr: [0.0075]
8 240 recon: [0.00688838133533527, 0.0011988295465278148] lr: [0.0075]
8 270 recon: [0.006348056562803892, 0.001242223260301122] lr: [0.0075]
8 300 recon: [0.005993623281862925, 0.001127199220128288] lr: [0.005625]
8 330 recon: [0.0063765902583937245, 0.0011995162146295985] lr: [0.005625]
8 360 recon: [0.006702081132045309, 0.001268890108924891] lr: [0.005625]
8 390 recon: [0.007684926351586409, 0.001958847858425072] lr: [0.005625]
9 0 recon: [0.005979666031678252, 0.0011609843078501667] lr: [0.005625]
9 30 recon: [0.006021822249210598, 0.0013645024181509957] lr: [0.005625]
9 60 recon: [0.006115107257893097, 0.0012721469812405197] lr: [0.005625]
9 90 recon: [0.006564391030374997, 0.0013385813607832565] lr: [0.005625]
9 120 recon: [0.00732597737259428, 0.0016051579663726172] lr: [0.005625]
9 150 recon: [0.006279757604782522, 0.001607427466881861

16 150 recon: [0.006078436407693872, 0.0017977248445774118] lr: [0.005625]
16 180 recon: [0.005923675835906718, 0.0016423391786896222] lr: [0.005625]
16 210 recon: [0.005726205585412582, 0.0014683156418460624] lr: [0.005625]
16 240 recon: [0.00626102579513073, 0.001935080213770247] lr: [0.005625]
16 270 recon: [0.005971420594670839, 0.0017161511089124164] lr: [0.005625]
16 300 recon: [0.005726296117943105, 0.0014537068008675016] lr: [0.005625]
16 330 recon: [0.006010742535637598, 0.0016313823636154954] lr: [0.005625]
16 360 recon: [0.006288715587697204, 0.0017874854481689825] lr: [0.005625]
16 390 recon: [0.0069997376863523714, 0.0021932454634922497] lr: [0.005625]
17 0 recon: [0.005734021053731914, 0.0014732830662646515] lr: [0.005625]
17 30 recon: [0.005738584459398825, 0.001723714223578123] lr: [0.005625]
17 60 recon: [0.005905063884786953, 0.0015397096942823256] lr: [0.005625]
17 90 recon: [0.006335833777624959, 0.001580832909965298] lr: [0.005625]
17 120 recon: [0.0070394437205218

24 120 recon: [0.006753641137264057, 0.0021367687585971086] lr: [0.005625]
24 150 recon: [0.005765080746531402, 0.0019785637209801883] lr: [0.005625]
24 180 recon: [0.005598179037891949, 0.001830037773395406] lr: [0.005625]
24 210 recon: [0.005416908734143648, 0.0016955455032309666] lr: [0.005625]
24 240 recon: [0.005936782482400056, 0.002176130575187403] lr: [0.005625]
24 270 recon: [0.005694658860482435, 0.0018846946905670104] lr: [0.005625]
24 300 recon: [0.005533192742021038, 0.0015814386124890483] lr: [0.005625]
24 330 recon: [0.005748420647747028, 0.0018073155542533597] lr: [0.005625]
24 360 recon: [0.006017730277445322, 0.0019923221990216515] lr: [0.005625]
24 390 recon: [0.00643039417542301, 0.002482981787311057] lr: [0.005625]
25 0 recon: [0.005512627419540638, 0.0016277332118157283] lr: [0.005625]
25 30 recon: [0.005473403189695718, 0.00188676823347349] lr: [0.005625]
25 60 recon: [0.005654595832773937, 0.0017085005509587397] lr: [0.005625]
25 90 recon: [0.0060152508366555415

32 30 recon: [0.00535872560049258, 0.001966565122741463] lr: [0.0031640625]
32 60 recon: [0.005529060563662075, 0.0018149452686372948] lr: [0.0031640625]
32 90 recon: [0.0058782881975445326, 0.0019330424388819079] lr: [0.0031640625]
32 120 recon: [0.006520206788098053, 0.0022893883074986327] lr: [0.0031640625]
32 150 recon: [0.005582627637878769, 0.0021377011328761087] lr: [0.0031640625]
32 180 recon: [0.005422491192758186, 0.00197427018720755] lr: [0.0031640625]
32 210 recon: [0.0052524130503095725, 0.0018460715939364767] lr: [0.0031640625]
32 240 recon: [0.00576940267598134, 0.0022684801007854766] lr: [0.0031640625]
32 270 recon: [0.0055301464433308994, 0.002033628918263333] lr: [0.0031640625]
32 300 recon: [0.005378597339213998, 0.0017241163676615288] lr: [0.0031640625]
32 330 recon: [0.005583860443726537, 0.0019178267795809452] lr: [0.0031640625]
32 360 recon: [0.0058603413227090685, 0.0020717086892226276] lr: [0.0031640625]
32 390 recon: [0.006113292492987194, 0.002687554648602953

39 240 recon: [0.005648372338453835, 0.002303206993152475] lr: [0.0031640625]
39 270 recon: [0.0054469367182539445, 0.0020250012646129225] lr: [0.0031640625]
39 300 recon: [0.005296409915912573, 0.001723928562618162] lr: [0.0031640625]
39 330 recon: [0.005481642820098079, 0.001956825132158237] lr: [0.0031640625]
39 360 recon: [0.005754876683545211, 0.002126402364538619] lr: [0.0031640625]
39 390 recon: [0.005895241287737291, 0.002720001115645832] lr: [0.0031640625]
40 0 recon: [0.005298398908124709, 0.0017618972973748564] lr: [0.0031640625]
40 30 recon: [0.005248159618880473, 0.0020151688928839806] lr: [0.0031640625]
40 60 recon: [0.005422400619768165, 0.0018449196919058703] lr: [0.0031640625]
40 90 recon: [0.005754640252410625, 0.001986237351059202] lr: [0.0031640625]
40 120 recon: [0.0063331683964983855, 0.0023416382914357506] lr: [0.0031640625]
40 150 recon: [0.0054735595498724594, 0.002138411523959717] lr: [0.0031640625]
40 180 recon: [0.005301280720606187, 0.001998841989159447] lr

47 30 recon: [0.005181561198483601, 0.0020539454918932073] lr: [0.0031640625]
47 60 recon: [0.005350161873496539, 0.0018890379318705606] lr: [0.0031640625]
47 90 recon: [0.005674576635074906, 0.002036084503398613] lr: [0.0031640625]
47 120 recon: [0.006185808625489587, 0.0024168374588236347] lr: [0.0031640625]
47 150 recon: [0.0053968430408047035, 0.0021781067489080813] lr: [0.0031640625]
47 180 recon: [0.005223857773213753, 0.0020440455605260964] lr: [0.0031640625]
47 210 recon: [0.005092306877487322, 0.0018987537088603654] lr: [0.0031640625]
47 240 recon: [0.005543422265016438, 0.002357568361221296] lr: [0.0031640625]
47 270 recon: [0.0053637690681406095, 0.002072707847811097] lr: [0.0031640625]
47 300 recon: [0.005208939433842628, 0.001779800170384108] lr: [0.0031640625]
47 330 recon: [0.005391906415855743, 0.0020095864026546976] lr: [0.0031640625]
47 360 recon: [0.005663937200401755, 0.00217866913237543] lr: [0.0031640625]
47 390 recon: [0.005752642527272367, 0.002783259729230143] 

In [255]:
# Fast weight estimation for all subjects

book = lrc()

A = book.reshape(book.shape[0], -1).permute(1,0)
AA = A.T@A
codes = []

for sub in range(x.shape[0]):
    B = get_sub_fc(x[sub]).reshape(-1, x.shape[-1])
    AB = A.T@B
    C,_,_,_ = torch.linalg.lstsq(AA+0.1*torch.eye(AA.shape[0]).float().cuda(),AB)
    codes.append(torch.from_numpy(C.detach().cpu().numpy()))
    if sub % 100 == 0:
        loss = mseLoss(A@C,B)**0.5
        print(f'Finished {sub} {loss}')
    
codes = torch.stack(codes)
print(codes.shape)

Finished 0 0.004010740667581558
Finished 100 0.004055102821439505
Finished 200 0.0038849436677992344
Finished 300 0.004113595932722092
Finished 400 0.004754922818392515
Finished 500 0.004039555322378874
Finished 600 0.005404890514910221
Finished 700 0.003230147063732147
Finished 800 0.005620813928544521
torch.Size([847, 300, 231])


In [256]:
# Reconstruct static FC from dynamic FC

import matplotlib.pyplot as plt

sub = 398

dynfc = get_sub_fc(x[sub])
dynfc = torch.mean(dynfc, dim=-1).detach().cpu().numpy()

statfc = np.corrcoef(ts[0][sub])

reconfc = lrw(sub, book)
reconfc = torch.mean(reconfc, dim=-1).detach().cpu().numpy()

A = book.reshape(book.shape[0], -1).permute(1,0)
reconfc2 = torch.mean((A.cpu()@codes[sub]), dim=1).reshape(264,264).numpy()

fig, ax = plt.subplots(1,4,figsize=(10,3))

print(np.mean((statfc-124*dynfc)**2)**0.5)
print(np.mean((statfc-124*reconfc)**2)**0.5)
print(np.mean((statfc-124*reconfc2)**2)**0.5)

ax[0].imshow(dynfc)
ax[1].imshow(statfc)
ax[2].imshow(reconfc)
ax[3].imshow(reconfc2)
fig.show()

RuntimeError: Can't call numpy() on Tensor that requires grad. Use tensor.detach().numpy() instead.

In [257]:
ntrain = 30

losses = []

for ntrain in [30,50,100,200,300,400,500,600,700,800]:

    codescuda = codes.float().cuda()
    xps = torch.mean(codescuda, dim=-1) #ps.reshape(ps.shape[0],-1)
    xps = torch.cat([xps, torch.ones(xps.shape[0], 1).float().cuda()], dim=1)
    xtr = xps[:ntrain]
    xt = xps[ntrain:]

    y = get_y(metadict, ['age'], subs)[0]
    y_t = torch.from_numpy(y).float().cuda()
    ytr = y_t[:ntrain]
    yt = y_t[ntrain:]

    w, _, _, _ = torch.linalg.lstsq(xtr.T@xtr + 0.01*torch.eye(301).float().cuda(), xtr.T@ytr)

#     print(torch.mean((ytr-xtr@w)**2)**0.5)
    losses.append(float(torch.mean((yt-xt@w)**2)**0.5))
    
print(losses)

[36.5264892578125, 36.16930389404297, 35.08573913574219, 34.39638137817383, 34.270389556884766, 33.73544692993164, 31.58016586303711, 31.929637908935547, 31.736000061035156, 31.088274002075195]


In [258]:
import itertools

import torch
import torch.nn as nn
import torch.nn.functional as F
    
class CNN(nn.Module):
    def __init__(self, x, dp=0.1):
        super(CNN, self).__init__()
        self.cnn1 = torch.nn.Conv2d(1,10,(x.shape[-2],4)).float().cuda()
        self.ap1 = torch.nn.AvgPool2d((1,x.shape[-1]-3))
        self.lin1 = torch.nn.Linear(10,1).float().cuda()
        self.dp = nn.Dropout(p=dp)
        
    def forward(self, ts):
        ts = self.dp(ts)
        y = F.relu(self.cnn1(ts))
        y = self.ap1(y)
        z = y.reshape(y.shape[0], -1).squeeze()
        y = self.lin1(z)
        return z, y.squeeze()
    
nEpochs = 1000
pPeriod = 200

ntrain = 30

losses = []

for ntrain in [30,50,100,200,300,400,500,600,700,800]:

    codescuda = codes.float().cuda()
    xps = codescuda.unsqueeze(1)
    xtr = xps[:ntrain]
    xt = xps[ntrain:]

    y = get_y(metadict, ['age'], subs)[0]
    y_t = torch.from_numpy(y).float().cuda()
    ytr = y_t[:ntrain]
    yt = y_t[ntrain:]

    cnn = CNN(xtr, dp=0)

    optim = torch.optim.Adam(cnn.parameters(), lr=1e-1, weight_decay=1e-1)
    sched = torch.optim.lr_scheduler.ReduceLROnPlateau(optim, patience=10, factor=0.9, eps=1e-7)

    for epoch in range(nEpochs):
        optim.zero_grad()
        z, yhat1 = cnn(xtr)
        loss = mseLoss(yhat1, ytr)
        loss.backward()
        optim.step()
        sched.step(loss)
#         if epoch % pPeriod == 0 or epoch == nEpochs-1:
#             print(f'{epoch} recon: {loss**0.5} {sched._last_lr}')

#     print('Complete')

    cnn.eval()
    z, yhat1 = cnn(xt)
    loss = mseLoss(yhat1, yt)**0.5
    losses.append(float(loss))
    
print(losses)

[38.116512298583984, 36.63888931274414, 35.6799430847168, 34.733787536621094, 33.64529800415039, 33.227699279785156, 31.140424728393555, 31.789806365966797, 31.55638313293457, 30.451705932617188]


In [262]:
ntrain = 800

losses = []

for ntrain in [30,50,100,200,300,400,500,600,700,800]:
    codescuda = codes.float().cuda()
    xps = codescuda.unsqueeze(1)
    xtr = xps[:ntrain]
    xt = xps[ntrain:]

    y = get_y(metadict, ['age'], subs)[0]
    y_t = torch.from_numpy(y).float().cuda()
    ytr = y_t[:ntrain]
    yt = y_t[ntrain:]

    mseLoss = nn.MSELoss()

    nEpochs = 500
    pPeriod = 100

    cnn = CNN(xtr, dp=0.1)
    sim = LatSim(1, torch.zeros(1,1,10), dp=0, edp=0.1, wInit=1e-4, dim=20, temp=1)

    optim = torch.optim.Adam(itertools.chain(cnn.parameters(), sim.parameters()), lr=1e-1, weight_decay=1e-2)
    sched = torch.optim.lr_scheduler.ReduceLROnPlateau(optim, patience=10, factor=0.9, eps=1e-7)

    for epoch in range(nEpochs):
        optim.zero_grad()
        z, yhat1 = cnn(xtr)
        yhat2 = sim(z.unsqueeze(1), [ytr])[0][0]
        loss = mseLoss(yhat2, ytr)
        loss.backward()
        optim.step()
        sched.step(loss)
    #     if epoch % pPeriod == 0 or epoch == nEpochs-1:
    #         print(f'{epoch} recon: {loss**0.5} {sched._last_lr}')

    # print('Complete')

    cnn.eval()
    sim.eval()
    z, yhat1 = cnn(xps)
    yhat2 = sim(z.unsqueeze(1), [y_t])[0][0][ntrain:]
    loss = mseLoss(yhat2, yt)**0.5
    losses.append(float(loss))
    
print(losses)

[36.68341064453125, 36.79133605957031, 34.97216796875, 34.43770217895508, 33.28044891357422, 31.87364387512207, 30.97648048400879, 33.13452911376953, 30.899599075317383, 29.04654884338379]


In [261]:
ntrain = 700

losses = []

for ntrain in [30,50,100,200,300,400,500,600,700,800]:
    codescuda = codes.float().cuda()
    xps = torch.mean(codescuda, dim=-1).unsqueeze(1)
    xtr = xps[:ntrain]
    xt = xps[ntrain:]

    y = get_y(metadict, ['age'], subs)[0]
    y_t = torch.from_numpy(y).float().cuda()
    ytr = y_t[:ntrain]
    yt = y_t[ntrain:]

    mseLoss = nn.MSELoss()

    nEpochs = 500
    pPeriod = 100

    sim = LatSim(1, torch.zeros(1,1,ncodes), dp=0.1, edp=0.1, wInit=1e-4, dim=2, temp=1)

    optim = torch.optim.Adam(sim.parameters(), lr=1e-1, weight_decay=1e-2)
    sched = torch.optim.lr_scheduler.ReduceLROnPlateau(optim, patience=10, factor=0.9, eps=1e-7)

    for epoch in range(nEpochs):
        optim.zero_grad()
        yhat = sim(xtr, [ytr])[0][0]
        loss = mseLoss(yhat, ytr)
        loss.backward()
        optim.step()
        sched.step(loss)
#         if epoch % pPeriod == 0 or epoch == nEpochs-1:
#             print(f'{epoch} recon: {loss**0.5} {sched._last_lr}')

#     print('Complete')

    sim.eval()
    yhat = sim(xps, [y_t])[0][0][ntrain:]
    loss = mseLoss(yhat, yt)**0.5
    losses.append(float(loss))
print(losses)

[36.64677429199219, 36.754432678222656, 35.20348358154297, 33.7934455871582, 33.08011245727539, 32.91068649291992, 30.93649673461914, 31.29697608947754, 31.2243709564209, 30.52267074584961]
